# Loop over fields and make html overview pages

In [1]:
import yaml
import glob
import numpy as np
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord
from pymoc import MOC
import healpy as hp

In [2]:
def field_to_sex(moc_file):
    "Take a moc and return the median ra dec"
    m = MOC(filename=moc_file)
    ra, dec = [],[]
    
    for cell in m.flattened(7):
        
        theta,phi = hp.pix2ang(2**7, cell, nest=True)

        dec.append((0.5*np.pi - theta) * 180/np.pi)
        ra.append(phi* 180/np.pi)
        
        

    c = SkyCoord(np.median(ra), np.median(dec),frame='icrs', unit='deg')
    
    return c.to_string('hmsdms').replace('h',' ').replace('m',' ').replace('d',' ').replace('s',' ')
    
#field_to_sex(yml['region'].replace('dmu_products','../../'))

In [3]:
meta_ymls = glob.glob('../../dmu32/*/meta_main.yml')

In [4]:
template = open('./html.template', 'r').read()

In [5]:
hedam='http://hedam.lam.fr/HELP/dataproducts'

In [6]:
overview=Table.read('../../dmu32/dr1_data_products_overview.csv')

In [7]:
overview

field,objects,dr1_file,dr1_file_hash,area_sq_degrees,file_size_bytes,file_size_readable,xid_objects,photoz_objects,cigale_objects
str18,int64,str81,str32,float64,int64,str9,int64,int64,int64
AKARI-NEP,531746,dmu_products/dmu32/dmu32_AKARI-NEP/data/AKARI-NEP_20180215.fits,5b33074e816785e05407952f03e78354,9.194732358779467,433440000,433.44 MB,31441,107228,1239
AKARI-SEP,844172,dmu_products/dmu32/dmu32_AKARI-SEP/data/AKARI-SEP_20180221.fits,56889960f654742f80329e8ebb0053d5,8.713306475131118,668652480,668.65 MB,108119,139059,566
Bootes,3398098,dmu_products/dmu32/dmu32_Bootes/data/Bootes_20190701.fits,6d2f18ef25a42999d805b258c751fde2,11.42815299095486,5556003840,5.56 GB,495159,1570512,38980
CDFS-SWIRE,2171051,dmu_products/dmu32/dmu32_CDFS-SWIRE/data/CDFS-SWIRE_20180613.fits,7bb01da6145d35ffa157b7c3ee0209ce,12.971246403717068,6059603520,6.06 GB,73511,136944,9308
COSMOS,2599374,dmu_products/dmu32/dmu32_COSMOS/data/COSMOS_20190402.fits,c900763f01cd327b84fafa3defe46151,5.083863478496816,10954031040,10.95 GB,25898,691502,15747
EGS,1412613,dmu_products/dmu32/dmu32_EGS/data/EGS_20180501.fits,74ce7e6bd7a982141d7558fad62b38df,3.566383275122158,4018965120,4.02 GB,223598,1182503,4159
ELAIS-N1,4026292,dmu_products/dmu32/dmu32_ELAIS-N1/data/ELAIS-N1_20171016.fits,97ccb7d86c92aa9e9ba0657c9e737276,13.507484555454763,6007360320,6.01 GB,269611,2714686,49985
ELAIS-N2,1783240,dmu_products/dmu32/dmu32_ELAIS-N2/data/ELAIS-N2_20180218.fits,09628a8b89fd5eeea27d6059988eb900,9.167479903991113,2321884800,2.32 GB,86591,120723,6798
ELAIS-S1,1655564,dmu_products/dmu32/dmu32_ELAIS-S1/data/ELAIS-S1_20180416.fits,e1e31bd8ae8b7646c47aaec6a40e2cf4,9.002940646885508,2231798400,2.23 GB,194276,1013582,25393


In [8]:
regions = yaml.load(open('../../dmu2/meta_main.yml', 'r'))

/Users/rs548/miniconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


In [9]:
for field_meta in meta_ymls:
    yml = yaml.load(open(field_meta, 'r'))
    print(yml['field'])
    
    description = yml['field']
    for f in regions['fields']:
        if f['name'] == yml['field']:
            description = f['description']
    #make html_table of surveys:
    survey_table = '<ul>' 
    for s in yml['surveys']:
        survey_table+='<li> <a href="https://github.com/H-E-L-P/dmu_products/tree/master/dmu0/dmu0_{}">{}</a></li>'.format(
            s, s
        ) 
    survey_table+= '</ul>'


    field_html = open('./' + yml['field'] + '.html', "w+")
    field_html.write(template.format(
        field=yml['field'],
        field_description=description,
        target = field_to_sex(yml['region'].replace('dmu_products','../../')), # '10 45 00 +45 00 00',
        fov=3*np.sqrt(overview[overview['field'] == yml['field']]['area_sq_degrees'][0]),
        area=round(overview[overview['field'] == yml['field']]['area_sq_degrees'][0],2),
        masterist_objects=overview[overview['field'] == yml['field']]['objects'][0],
        xid_objects=overview[overview['field'] == yml['field']]['xid_objects'][0],
        photoz_objects=overview[overview['field'] == yml['field']]['photoz_objects'][0],
        cigale_objects=overview[overview['field'] == yml['field']]['cigale_objects'][0],
        median_g_depth='PLACEHOLDER',
        median_k_depth='PLACEHOLDER',
        median_i1_depth='PLACEHOLDER',
        median_250_depth='PLACEHOLDER',
        best_phot=hedam + '/dmu32/dmu32_AllSky/data/{}_best_phot_20201111.fits'.format(yml['field']),
        master_list=hedam+yml['final'],
        blind_list=(
            hedam+
            '/dmu22/dmu22_{}/data/dmu22_XID+SPIRE_{}_BLIND_Matched_MF.fits'.format(
                yml['field'],yml['field']
            )
        ),
        spire_250=hedam+'/dmu19/dmu19_HELP-SPIRE-maps/data/{}_SPIRE250_v1.0.fits'.format(yml['field']),
        spire_350=hedam+'/dmu19/dmu19_HELP-SPIRE-maps/data/{}_SPIRE350_v1.0.fits'.format(yml['field']), 
        spire_500=hedam+'/dmu19/dmu19_HELP-SPIRE-maps/data/{}_SPIRE500_v1.0.fits'.format(yml['field']),
        photoz_file=hedam+yml['photoz'],
        cigale_file=hedam+yml['cigale'],
        seds=hedam+'/dmu28/dmu28_{}/data/zphot/SEDs/'.format(yml['field']),
        moc=hedam+yml['region'],
        survey_table=survey_table
    ))
    field_html.close()

/Users/rs548/miniconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


COSMOS
ELAIS-S1
GAMA-09
Lockman-SWIRE
ELAIS-N2
Herschel-Stripe-82
SA13
GAMA-12
GAMA-15
AKARI-SEP
HATLAS-SGP
xFLS
CDFS-SWIRE
SSDF
XMM-LSS
SPIRE-NEP
AKARI-NEP
ELAIS-N1
HATLAS-NGP
XMM-13hr
Bootes
HDF-N
EGS
